In [1]:
import logging
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
import numpy as np

# from keras.models import Sequential
# from keras.layers import Conv1D, MaxPooling1D
# from keras.layers import Activation, Dropout, Flatten, Dense

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Load data from the data file
Column names also added
Probable colomn names
Gender, Age, Debt, Married, BankCustomer, EducationLevel, Ethnicity, YearsEmployed, PriorDefault, Employed, CreditScore, DriversLicense, Citizen, ZipCode, Income and ApprovalStatus

In [2]:
columns = ['A1', 'A2','A3','A4', 'A5', 'A6', 'A7', 'A8','A9', 'A10', 'A11','A12', 'A13', 'A14', 'A15', 'label']
df = pd.read_csv('../data/crx.data', names=columns)
df.replace('?', np.nan, inplace=True)
df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,label
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


In [3]:
df.describe()

,A3,A8,A11,A15
count,690.000000,690.000000,690.00000,690.000000
mean,4.758725,2.223406,2.40000,1017.385507
std,4.978163,3.346513,4.86294,5210.102598
min,0.000000,0.000000,0.00000,0.000000
25%,1.000000,0.165000,0.00000,0.000000
50%,2.750000,1.000000,0.00000,5.000000
75%,7.207500,2.625000,3.00000,395.500000
max,28.000000,28.500000,67.00000,100000.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      678 non-null    object 
 1   A2      678 non-null    object 
 2   A3      690 non-null    float64
 3   A4      684 non-null    object 
 4   A5      684 non-null    object 
 5   A6      681 non-null    object 
 6   A7      681 non-null    object 
 7   A8      690 non-null    float64
 8   A9      690 non-null    object 
 9   A10     690 non-null    object 
 10  A11     690 non-null    int64  
 11  A12     690 non-null    object 
 12  A13     690 non-null    object 
 13  A14     677 non-null    object 
 14  A15     690 non-null    int64  
 15  label   690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


### Remove missing values
+ Replace '?' with NaN
+ Then replace all numerical NAN values with the mean
+ Replace categorical values with the most occured element

In [5]:

# print(df.tail(20))
print(df.info())
print("NaN values: {}".format(df.isnull().values.sum()))
df = df.fillna(df.mean())
# print(df.tail(20))
# print("NaN values: {}".format(df.isnull().values.sum()))
df['A1'].value_counts()

for col in df.columns:
    # Check if the column is of object type
    if df[col].dtypes == 'object':
        # Impute with the most frequent value
        df[col] = df[col].fillna(df[col].value_counts().index[0])
print(df.info())       
print("NaN values: {}".format(df.isnull().values.sum()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      678 non-null    object 
 1   A2      678 non-null    object 
 2   A3      690 non-null    float64
 3   A4      684 non-null    object 
 4   A5      684 non-null    object 
 5   A6      681 non-null    object 
 6   A7      681 non-null    object 
 7   A8      690 non-null    float64
 8   A9      690 non-null    object 
 9   A10     690 non-null    object 
 10  A11     690 non-null    int64  
 11  A12     690 non-null    object 
 12  A13     690 non-null    object 
 13  A14     677 non-null    object 
 14  A15     690 non-null    int64  
 15  label   690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB
None
NaN values: 67
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---

### Convert none neumeric data into neumeric values

In [6]:

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Iterate over all the values of each column and extract their dtypes
for col in df.columns:
    # Compare if the dtype is object
    if df[col].dtype=='object':
    # Use LabelEncoder to do the numeric transformation
        df[col]=le.fit_transform(df[col])
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,label
0,1,156,0.000,1,0,12,7,1.25,1,1,1,0,0,68,0,0
1,0,328,4.460,1,0,10,3,3.04,1,1,6,0,0,11,560,0
2,0,89,0.500,1,0,10,3,1.50,1,0,0,0,0,96,824,0
3,1,125,1.540,1,0,12,7,3.75,1,1,5,1,0,31,3,0
4,1,43,5.625,1,0,12,7,1.71,1,0,0,0,2,37,0,0


### Drop the features that are not usable
As we have a good understanding about the data

1. Gender
2. Age
3. Debt
4. Married
5. BankCustomer
6. EducationLevel
7. Ethnicity
8. YearsEmployed
9. PriorDefault
10. Employed
11. CreditScore
12. DriversLicense
13. Citizen
14. ZipCode
15. Income
16. ApprovalStatus

Looking at these data its clear that 
+ Zipcode
+ Drivers License

IS not relevent for this classification

In [7]:
df.drop(['A12', 'A14'],axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      690 non-null    int64  
 1   A2      690 non-null    int64  
 2   A3      690 non-null    float64
 3   A4      690 non-null    int64  
 4   A5      690 non-null    int64  
 5   A6      690 non-null    int64  
 6   A7      690 non-null    int64  
 7   A8      690 non-null    float64
 8   A9      690 non-null    int64  
 9   A10     690 non-null    int64  
 10  A11     690 non-null    int64  
 11  A13     690 non-null    int64  
 12  A15     690 non-null    int64  
 13  label   690 non-null    int64  
dtypes: float64(2), int64(12)
memory usage: 75.6 KB


### Normalize all the neumerical data

In [8]:
from sklearn.preprocessing import MinMaxScaler

dfn = df.values

# Segregate features and labels into separate variables
X,y = dfn[:,0:13], dfn[:,13]


# Instantiate MinMaxScaler and use it to rescale
scaler = MinMaxScaler(feature_range=(0,1))
rescaledX = scaler.fit_transform(X)

print(y.shape)
print(rescaledX.shape)

(690,)
(690, 13)


### Create the model
It takes input as 
+ No of dense neurons
+ Activation functions
+ Regularizers

In [26]:
def create_model_v2(layer_info):
    inputs=tf.keras.Input(shape=13)
    x = inputs
    for li in layer_info['layers']:
        x = tf.keras.layers.Dense(li[0], activation=li[1], kernel_regularizer=li[2])(x)
                                  
    output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)

    model = tf.keras.Model(inputs=inputs, outputs=output)
    if layer_info['verbose'] == 1:
        logging.info(model.summary())

    # Compile model
    opt = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=layer_info['optimizer'], loss=layer_info['loss'], metrics=['accuracy'])

    return model

### K fold cross validation
This will try 5 fold cross validation


In [10]:
from sklearn.metrics import f1_score
def train_with_kfold(model):
    n_folds = 5
    f1_score_list = []
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    for train_ix, test_ix in kfold.split(rescaledX):
        X_train, X_test, y_train, y_test = rescaledX[train_ix],rescaledX[test_ix], y[train_ix], y[test_ix]
        history = model_v2.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=0)
        y_pred = np.where(model_v2.predict(X_test) > 0.5, 1, 0)
        f1 = f1_score(y_test, y_pred , average="macro")
        f1_score_list.append(float("{:2.2f}".format(f1*100)))
        
    print("F1 score: {}".format(f1_score_list))
    print("Average F1: {:.3f}".format(np.mean(f1_score_list)))

In [11]:
print("Average F1: {:2.3f}".format(np.mean([1,2,3,4,5])*100))

Average F1: 300.000


## Try different number of Layers

In [27]:
layer_info={"layers":[
    [5, tf.nn.relu, None]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

F1 score: [87.68, 88.1, 86.2, 80.35, 86.91]
Average F1: 85.848


In [13]:
layer_info={"layers":[
    [5, tf.nn.relu, None],
    [5, tf.nn.relu, None]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

[5, <function relu at 0x7f89a86947a0>, <tensorflow.python.keras.regularizers.L1L2 object at 0x7f8904795990>]
[5, <function relu at 0x7f89a86947a0>, <tensorflow.python.keras.regularizers.L1L2 object at 0x7f8904195d50>]
F1 score: [87.65, 88.76, 85.43, 82.48, 89.08]
Average F1: 86.680


In [14]:
layer_info={"layers":[
    [5, tf.nn.relu, None],
    [5, tf.nn.relu, None],
    [5, tf.nn.relu, None]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

[5, <function relu at 0x7f89a86947a0>, <tensorflow.python.keras.regularizers.L1L2 object at 0x7f890469b6d0>]
[5, <function relu at 0x7f89a86947a0>, <tensorflow.python.keras.regularizers.L1L2 object at 0x7f890469bf50>]
[5, <function relu at 0x7f89a86947a0>, <tensorflow.python.keras.regularizers.L1L2 object at 0x7f890469bf90>]
F1 score: [89.12, 87.33, 85.4, 81.72, 86.14]
Average F1: 85.942


## Try different number of neurons
5,10,15,20,25

In [28]:
layer_info={"layers":[
    [5, tf.nn.relu, None],
    [5, tf.nn.relu, None]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

F1 score: [88.4, 87.33, 85.43, 81.81, 89.1]
Average F1: 86.414


In [29]:
layer_info={"layers":[
    [10, tf.nn.relu, None],
    [10, tf.nn.relu, None]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

F1 score: [86.91, 91.71, 85.31, 83.23, 89.8]
Average F1: 87.392


In [30]:
layer_info={"layers":[
    [15, tf.nn.relu, None],
    [15, tf.nn.relu, None]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

F1 score: [89.08, 90.14, 88.29, 84.6, 89.82]
Average F1: 88.386


In [31]:
layer_info={"layers":[
    [20, tf.nn.relu, None],
    [20, tf.nn.relu, None]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

F1 score: [89.03, 90.93, 89.72, 82.52, 90.54]
Average F1: 88.548


In [32]:
layer_info={"layers":[
    [25, tf.nn.relu, None],
    [25, tf.nn.relu, None]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

F1 score: [88.32, 90.2, 90.43, 83.23, 89.82]
Average F1: 88.400


In [33]:
layer_info={"layers":[
    [30, tf.nn.relu, None],
    [30, tf.nn.relu, None]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

F1 score: [89.03, 89.48, 88.16, 85.36, 92.02]
Average F1: 88.810


## Try different activation functions

In [15]:
layer_info={"layers":[
    [5, tf.nn.relu, None]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

[5, <function relu at 0x7f89a86947a0>, <tensorflow.python.keras.regularizers.L1L2 object at 0x7f890417c690>]
F1 score: [88.4, 88.82, 85.46, 81.84, 85.48]
Average F1: 86.000


In [16]:
layer_info={"layers":[
    [5, tf.nn.softmax, None]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

[5, <function softmax_v2 at 0x7f89a6ff3b90>, <tensorflow.python.keras.regularizers.L1L2 object at 0x7f89041c2810>]
F1 score: [87.68, 88.82, 87.63, 81.06, 86.89]
Average F1: 86.416


In [17]:
layer_info={"layers":[
    [5, tf.nn.elu, None]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

[5, <function elu at 0x7f89a8743d40>, <tensorflow.python.keras.regularizers.L1L2 object at 0x7f89042d80d0>]
F1 score: [88.4, 88.1, 83.31, 81.1, 87.63]
Average F1: 85.708


In [18]:
layer_info={"layers":[
    [5, tf.nn.relu6, None]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

[5, <function relu6 at 0x7f89a6ff3830>, <tensorflow.python.keras.regularizers.L1L2 object at 0x7f890454f0d0>]
F1 score: [88.4, 88.1, 85.48, 80.35, 87.66]
Average F1: 85.998


In [19]:
layer_info={"layers":[
    [5, tf.nn.leaky_relu, None]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

[5, <function leaky_relu at 0x7f89a6ff38c0>, <tensorflow.python.keras.regularizers.L1L2 object at 0x7f89046a72d0>]
F1 score: [87.68, 88.87, 84.03, 81.1, 87.66]
Average F1: 85.868


## Try different Losses - [Probabilistic]

In [20]:
layer_info={"layers":[
    [5, tf.nn.leaky_relu, None ]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

[5, <function leaky_relu at 0x7f89a6ff38c0>, <tensorflow.python.keras.regularizers.L1L2 object at 0x7f88e0479490>]
F1 score: [87.68, 88.87, 83.31, 81.1, 87.63]
Average F1: 85.718


In [21]:
layer_info={"layers":[
    [5, tf.nn.leaky_relu, None]
           ],
            "loss": "sparse_categorical_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

[5, <function leaky_relu at 0x7f89a6ff38c0>, <tensorflow.python.keras.regularizers.L1L2 object at 0x7f89045b1210>]


/home/dulanj/Environments/py376/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':


F1 score: [31.34, 28.12, 29.95, 32.02, 32.35]
Average F1: 30.756


In [22]:
layer_info={"layers":[
    [5, tf.nn.leaky_relu, None ]
           ],
            "loss": "poisson",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

[5, <function leaky_relu at 0x7f89a6ff38c0>, <tensorflow.python.keras.regularizers.L1L2 object at 0x7f89800c7e10>]
F1 score: [87.68, 86.55, 84.76, 81.06, 86.89]
Average F1: 85.388


In [23]:
layer_info={"layers":[
    [5, tf.nn.leaky_relu, None ]
           ],
            "loss": "categorical_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

[5, <function leaky_relu at 0x7f89a6ff38c0>, <tensorflow.python.keras.regularizers.L1L2 object at 0x7f89044b5b10>]
F1 score: [59.2, 51.68, 54.65, 49.44, 53.9]
Average F1: 53.774



## Question 4
 Take the best network you found in Q2 above and retrain it with the addition of
regularization to the loss function. Try L1 and L2 regularization. For each training set in your 5-
fold cross validation, use part of it as a validation set to try out different values of λ to find the
best trade off between the loss function and the regularizing term. Report the final average F1
score on the test sets of the 5-fold cross validation and comment on whether regularization
helped or not.





In [38]:
layer_info={"layers":[
    [15, tf.nn.relu, tf.keras.regularizers.l1(0.01)],
    [15, tf.nn.relu, tf.keras.regularizers.l1(0.01)]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

F1 score: [86.96, 88.1, 84.76, 81.06, 86.2]
Average F1: 85.416


In [39]:
layer_info={"layers":[
    [15, tf.nn.relu, tf.keras.regularizers.l2(0.01)],
    [15, tf.nn.relu, tf.keras.regularizers.l2(0.01)]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

F1 score: [86.23, 88.1, 84.74, 81.1, 86.89]
Average F1: 85.412


In [42]:
layer_info={"layers":[
    [15, tf.nn.relu, tf.keras.regularizers.l1_l2(0.01)],
    [15, tf.nn.relu, tf.keras.regularizers.l1_l2(0.01)]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

F1 score: [86.96, 87.44, 84.04, 82.58, 86.2]
Average F1: 85.444


In [41]:
layer_info={"layers":[
    [15, tf.nn.relu, None ],
    [15, tf.nn.relu, None ]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
model_v2 = create_model_v2(layer_info)
train_with_kfold(model_v2)

F1 score: [89.8, 90.98, 86.78, 83.97, 91.99]
Average F1: 88.704


## Different lambda values for 

In [46]:
for i in range(10):
    _lambda = 0.1**i
    print("Lambda: {}".format(_lambda))
    layer_info={"layers":[
    [15, tf.nn.relu, tf.keras.regularizers.l1_l2(_lambda)],
    [15, tf.nn.relu, tf.keras.regularizers.l1_l2(_lambda)]
           ],
            "loss": "binary_crossentropy",
            "optimizer": "adam",
            "verbose": 0
           }
    model_v2 = create_model_v2(layer_info)
    train_with_kfold(model_v2)

Lambda: 1.0
F1 score: [35.21, 37.84, 36.41, 34.6, 34.29]
Average F1: 35.670
Lambda: 0.1
F1 score: [35.21, 37.84, 36.41, 34.6, 34.29]
Average F1: 35.670
Lambda: 0.010000000000000002
F1 score: [86.96, 88.1, 84.76, 81.84, 86.2]
Average F1: 85.572
Lambda: 0.0010000000000000002
F1 score: [86.96, 87.39, 84.76, 81.1, 86.14]
Average F1: 85.270
Lambda: 0.00010000000000000002
F1 score: [86.96, 87.39, 84.03, 81.1, 85.4]
Average F1: 84.976
Lambda: 1.0000000000000003e-05
F1 score: [87.68, 88.1, 84.76, 81.84, 86.14]
Average F1: 85.704
Lambda: 1.0000000000000004e-06
F1 score: [86.95, 88.1, 84.76, 81.84, 86.14]
Average F1: 85.558
Lambda: 1.0000000000000004e-07
F1 score: [86.96, 88.1, 84.76, 80.35, 85.4]
Average F1: 85.114
Lambda: 1.0000000000000005e-08
F1 score: [87.68, 87.33, 84.76, 81.1, 85.4]
Average F1: 85.254
Lambda: 1.0000000000000005e-09
F1 score: [86.96, 87.39, 84.76, 81.84, 86.14]
Average F1: 85.418


In [ ]:
==============================================================================================

In [ ]:
def create_model(verbose=False):
    
    model = tf.keras.models.Sequential()
  
    if verbose:
        print('Network configuration ',neuronCountInEachLayer)
    model.add(tf.keras.layers.Dense(neuronCountInEachLayer[0], input_dim=13, activation = activationFuncEachLayer[0], kernel_regularizer=regularizerFunc)) # First Layer
    
    for x in range(1, depthOfNetwork-1):
        
        model.add(tf.keras.layers.Dense(neuronCountInEachLayer[x], activation = activationFuncEachLayer[x],kernel_regularizer=regularizerFunc))         # Second layer onwards
 
    model.add(tf.keras.layers.Dense(neuronCountInEachLayer[depthOfNetwork-1], activation = activationFuncEachLayer[depthOfNetwork-1]))  # Output layer
    
    model.compile(loss = lossFunction , optimizer = 'adam' , metrics = ['accuracy'] ) 
        
    return model  

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold

def evaluateTheModel(verbose=False):
    n_split=5
    f1_scores = []

    for train_index,test_index in StratifiedKFold(n_split).split(X, y):      # StratifiedKFold, KFold
        x_train,x_test=X[train_index],X[test_index]
        y_train,y_test=y[train_index],y[test_index]

        model=create_model(verbose)
        model.fit(x_train, y_train,epochs=100, verbose=0)
        evaluationMetrics = model.evaluate(x_test,y_test, verbose=0)
        
        if verbose:
            print('Model evaluation ',evaluationMetrics)   # This returns metric values for the evaluation

        y_pred = np.where(model.predict(x_test) > 0.5, 1, 0)
        print(model.predict(x_test))
        f1 = f1_score(y_test, y_pred , average="macro")

        if verbose:
            print('F1 score is ', f1)
        
        f1_scores.append(f1)
    
    return np.mean(f1_scores)

In [ ]:
depthOfNetwork = 2
neuronCountInEachLayer = [2, 1]                                 # try different depth and width
activationFuncEachLayer = ['sigmoid', 'sigmoid']            # try values relu, sigmoid, talh
lossFunction = 'binary_crossentropy'                                # try values binary_crossentropy, mean_squared_error
regularizerFunc = tf.keras.regularizers.l2(0)                       # try l1 and l2 with different lambda

evaluateTheModel(True)